In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    if gpu_num!="":
        print(f'\nUsing GPU {gpu_num}\n')
    else:
        print('\nUsing CPU\n')
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"

# Import Sionna
try:
    import sionna.phy
    import sionna.sys
except ImportError as e:
    import sys
    if 'google.colab' in sys.modules:
       # Install Sionna in Google Colab
       print("Installing Sionna and restarting the runtime. Please run the cell again.")
       os.system("pip install sionna")
       os.kill(os.getpid(), 5)
    else:
       raise e

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

In [4]:
# Additional external libraries
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np

# Sionna components
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, \
    RadioMapSolver, PathSolver, subcarrier_frequencies, Camera
from sionna.phy import config
from sionna.phy.mimo import StreamManagement
from sionna.phy.ofdm import ResourceGrid, RZFPrecodedChannel, LMMSEPostEqualizationSINR
from sionna.phy.constants import BOLTZMANN_CONSTANT
from sionna.phy.nr.utils import decode_mcs_index
from sionna.phy.utils import log2, dbm_to_watt, lin_to_db
from sionna.sys import PHYAbstraction, OuterLoopLinkAdaptation, \
    PFSchedulerSUMIMO, downlink_fair_power_control
from sionna.sys.utils import spread_across_subcarriers

# Internal computational precision
config.precision = 'single'  # 'single' or 'double'

# Set random seed for reproducibility
config.seed = 48

# Toggle to False to use the preview widget
# instead of rendering for scene visualization
no_preview = True

In [5]:
# Number of simulated slots
num_slots = 200

# Time/frequency resource grid
carrier_frequency = 3.5e9
num_subcarriers = 128
subcarrier_spacing = 30e3
num_ofdm_symbols = 12

# Start/end 3D position of the users
# You can try and change these values to see how the system behaves
ut_pos_start = np.array([[-25,   0, 1.5],  # user 1
                         [24,  55, 1.5],  # user 2
                         [88,   0, 1.5]])  # user 3

ut_pos_end = np.array([[-25,  30, 1.5],    # user 1
                       [24,  15, 1.5],    # user 2
                       [65,    0, 1.5]])   # user 3

# Base station position and look-at direction
bs_pos = np.array([32.5, 10.5, 23])
bs_look_at = np.array([22, -8, 0])

# Number of users and base stations
num_bs = 1
num_ut = ut_pos_start.shape[0]

# Environment temperature
temperature = 294  # [K]

# BLER target value
bler_target = .1  # in [0; 1]

# MCS table index
mcs_table_index = 1

# Base station transmit power
# Low power is sufficient here thanks to lack of interference
bs_power_dbm = 10  # [dBm]
bs_power_watt = dbm_to_watt(bs_power_dbm)  # [W]

# Number of antennas at the transmitter and receiver
num_bs_ant = num_ut
num_ut_ant = 1

In [6]:
# Number of streams per user and base station
num_streams_per_ut = num_ut_ant
num_streams_per_bs = num_ut_ant * num_ut

# Noise power per subcarrier
no = BOLTZMANN_CONSTANT * temperature * subcarrier_spacing

# Stream management: Rx to Tx association
# Since there is only a single BS, all UTs are connected to it
rx_tx_association = np.ones([num_ut, num_bs])
stream_management = StreamManagement(rx_tx_association, num_streams_per_bs)

# OFDM resource grid
resource_grid = ResourceGrid(num_ofdm_symbols=num_ofdm_symbols,
                             fft_size=num_subcarriers,
                             subcarrier_spacing=subcarrier_spacing,
                             num_tx=num_ut,
                             num_streams_per_tx=num_streams_per_ut)

# Subcarrier frequencies
frequencies = subcarrier_frequencies(num_subcarriers=num_subcarriers,
                                     subcarrier_spacing=subcarrier_spacing)

In [ ]:
# Configure Sionna RT
p_solver = PathSolver()

# Load the scene
scene = load_scene(sionna.rt.scene.simple_street_canyon)

# Set the scene parameters
scene.frequency = carrier_frequency
scene.bandwidth = num_subcarriers*subcarrier_spacing
scene.tx_array = PlanarArray(
    num_rows=num_bs_ant, num_cols=1, pattern="tr38901", polarization='V')
scene.rx_array = PlanarArray(
    num_rows=1, num_cols=num_ut_ant, pattern="dipole", polarization='V')

# Add base station to the scene
scene.add(Transmitter(f"bs0", position=bs_pos, look_at=bs_look_at,
                      power_dbm=bs_power_dbm, display_radius=3))

# Emulate moving users by placing multiple receivers along a straight line
step = (ut_pos_end - ut_pos_start) / (num_slots - 1)

# We add all users at all future positions at once
for slot in range(num_slots):
    pos_curr = ut_pos_start + slot * step
    # Add users
    for ut in range(num_ut):
        scene.add(Receiver(f"ut{ut}_slot{slot}", position=pos_curr[ut, :],
                  display_radius=1, color=[0, 0, 0]))